In [3]:
import torch
import torch.nn.functional as F

num_nodes = 10

sparsity = 0.3

adjacency_matrix = torch.rand(num_nodes, num_nodes)
adjacency_matrix = (adjacency_matrix < sparsity).float()


adjacency_matrix = adjacency_matrix * (1 - torch.eye(num_nodes))

print("Random Adjacency Matrix:")
print(adjacency_matrix)

Random Adjacency Matrix:
tensor([[0., 1., 0., 1., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 1., 0., 1., 1., 0.],
        [0., 1., 0., 0., 0., 1., 1., 0., 0., 0.],
        [1., 0., 1., 1., 0., 0., 0., 0., 0., 1.],
        [1., 0., 1., 1., 1., 1., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 1., 0., 0., 0., 0., 1.],
        [1., 0., 1., 0., 0., 1., 1., 1., 1., 0.]])


In [18]:
from model import CongestionLearnableEmbedding, CongestionWrapperEncoder, CustomLearnableEmbedding
import torch
from parameters import CONGESTION_EMBEDDING_DIM, CONGESTION_SPACE_CARDINAL

num_nodes = 10

cong = torch.randint(1, 10, size=(10, num_nodes)).unsqueeze(0)
print(cong)

edge_index = torch.randint(high=num_nodes-1, size=(2, num_nodes**2//10))
print(edge_index)
x = CongestionWrapperEncoder(CustomLearnableEmbedding, num_nodes, CONGESTION_SPACE_CARDINAL, CONGESTION_EMBEDDING_DIM, CONGESTION_EMBEDDING_DIM, 1, 0.1)
# print(x(cong, edge_index))
print(x(cong, edge_index).shape)

ImportError: cannot import name 'CongestionLearnableEmbedding' from 'model' (/Users/isaigordeev/Desktop/congestion/models/model.py)

In [1]:
from model import CongestionLearnableEmbedding, CongestionWrapperEncoder, CongestionModel, CongestionTransformerDecoder
import torch
from parameters import CONGESTION_EMBEDDING_DIM
num_nodes = 10
cong = torch.randint(1, 10, size=(10, num_nodes)).unsqueeze(0)
print(cong)

edge_index = torch.randint(high=num_nodes-1, size=(2, num_nodes**2//10))

# x = CongestionWrapperEncoder(CongestionLearnableEmbedding, num_nodes, CONGESTION_EMBEDDING_DIM, CONGESTION_EMBEDDING_DIM, 1, 0.1)
# print(x(cong, edge_index))
# x(cong, edge_index).shape

x = CongestionModel(CongestionLearnableEmbedding, CongestionWrapperEncoder, CongestionTransformerDecoder, None, 10, 5, 5, 1, 0.1)(cong, edge_index)
x

tensor([[[5, 3, 6, 1, 5, 7, 2, 8, 3, 6],
         [8, 9, 9, 6, 7, 4, 3, 3, 6, 5],
         [8, 6, 4, 4, 9, 2, 6, 5, 2, 6],
         [3, 7, 4, 2, 5, 2, 9, 4, 9, 4],
         [4, 5, 2, 2, 4, 4, 9, 8, 9, 7],
         [6, 8, 7, 4, 5, 1, 3, 3, 6, 4],
         [3, 6, 9, 4, 8, 6, 4, 4, 8, 8],
         [8, 1, 3, 6, 7, 6, 7, 1, 7, 4],
         [3, 3, 1, 6, 8, 6, 7, 3, 7, 1],
         [4, 4, 5, 9, 9, 2, 2, 4, 6, 4]]])


TypeError: empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [1]:
import torch.nn as nn
from torch_geometric.nn import GCNConv

class LearnableNodeGCN(nn.Module):
    def __init__(self, in_features, out_features, num_learnable_features):
        super(self).__init__()

        self.node_embedding = nn.Linear(in_features, num_learnable_features)
        self.conv = GCNConv(num_learnable_features + in_features, out_features)

    def forward(self, x, edge_index):
        node_embeddings = self.node_embedding(x)
        x = torch.cat((node_embeddings, x), dim=1)
        x = self.conv(x, edge_index)

        return x


In [2]:
import torch
from torch_geometric.nn import GCNConv
from torchinfo import summary

model = LearnableNodeGCN(3, 5, 2)
device = torch.device("cpu" if not torch.cuda.is_available() else "cuda")
model.to(device)

# Print a summary of the model
summary(model, input_names=["x", "edge_index"])


Layer (type:depth-idx)                   Param #
LearnableNodeGCN                         --
├─Linear: 1-1                            8
├─GCNConv: 1-2                           5
│    └─SumAggregation: 2-1               --
│    └─Linear: 2-2                       25
Total params: 38
Trainable params: 38
Non-trainable params: 0

In [6]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv


edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)
x = torch.randn(3, 16)
print(x.grad)

data = Data(x=x, edge_index=edge_index)
print(x.grad_fn)

class GNNModel(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GNNModel, self).__init__()
        self.conv = GCNConv(in_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv(x, edge_index)
        return x


model = GNNModel(in_channels=16, out_channels=10)
device = torch.device("cpu" if not torch.cuda.is_available() else "cuda")
model.to(device)

# Print a summary of the model
summary(model, input_names=["x", "edge_index"])


None
None


Layer (type:depth-idx)                   Param #
GNNModel                                 --
├─GCNConv: 1-1                           10
│    └─SumAggregation: 2-1               --
│    └─Linear: 2-2                       160
Total params: 170
Trainable params: 170
Non-trainable params: 0

In [13]:
import torch
from torch_geometric.data import Data


embeddings = nn.Embedding(7, 4)


# Create a Data object with node features
x = torch.tensor([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]], requires_grad=True)
edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)

x = torch.tensor(x, x)
data = Data(x=x, edge_index=edge_index)

# Perform some operation on the node features
output = data.x.sum()

# Perform backpropagation
output.backward()

# Access gradients
gradients = data.x.grad

print("Node features:", data.x)
print("Gradients:", gradients)


TypeError: tensor() takes 1 positional argument but 2 were given

In [19]:
torch.randn(1, 1, 5)

tensor([[[ 0.7733, -1.7869,  0.7220,  0.4980,  0.2973]]])